## 관련 환경설정 설치

In [ ]:
pip install -r requirements.txt

## Gemma 모델 준비

# 허깅 페이스에 로그인
from huggingface_hub import login

# Hugging Face API 토큰을 입력합니다.
api_token = "Your_Huggingface_Token"
login(api_token)

In [ ]:
# Gemma-2b-it 모델과 토크나이저 다운로드
import torch
import wandb

from sklearn.model_selection import train_test_split

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    Trainer
)
from transformers.intergrations import WandbCallback
from trl import DataCollatorForCompletionOnlyLM
import evaluate

# 모델과 토크나이저 불러오기
model_name = "googld/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_name,
    use_cache =False, # 빨리 불러오지만 메모리 사용량 증가
    device_map="auto",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    attn_implementation="eager",
)
tokenizer = AutoTokenizer.from_pretrained(model_name) # 해당 모델과 같이 학습시킨 토크나이저 로드

In [ ]:
# 데이터셋 준비
import datasets
dataset = datasets.load_dataset("jaehy12/news3")
print(dataset.keys())
element = dataset["train"][1] # train dataset의 1번째 data
element

In [ ]:
# 키워드 추출 기능 task 확인하기
input_text = """다음 텍스트를 한국어로 간단히 요약해주세요:\n부산의 한 왕복 2차선 도로에서 역주행 사고로 배달 오토바이 운전자인 고등학생이 숨지는 사고가 발생했다.
유족은 '가해자가 사고 후 곧바로 신고하지 않고 늑장 대응해 피해를 키웠다'고 주장하고 있다.
11일 부산진경찰서는 교통사고처리특례법(교통사고처리법)상 업무상 과실치사 혐의로 지난 3일 A(59)씨를 검찰에 불구속 송치했다고 밝혔다.
A씨는 교통사고처리법상 12대 중과실에 해당되는 '중앙선 침범'으로 역주행 교통사고를 일으킨 혐의를 받는다.
경찰에 따르면 스포츠유틸리티차량(SUV) 운전자 A씨는 5월 19일 밤 11시 50분쯤 부산진구 가야고가교 밑 도로에서 중앙선을 넘어 역주행으로 140m를 달려
반대편 차선의 오토바이 운전자 조모(16)군을 들이받았다. 조군은 원동기장치자전거 면허를 취득한 상태였고 헬멧도 쓰고 있었지만 크게 다쳤다.
사고 당일 수술을 받았으나 얼마 후 2차 뇌출혈로 뇌사 판정이 내려졌고, 사고 발생 약 한 달 만인 지난달 16일 끝내 사망했다.
사고를 낸 A씨는 술을 마시거나 약물을 복용한 상태에서 운전하지는 않은 것으로 조사됐다.
경찰 관계자는 'A씨가 자신이 정주행을 하고 오토바이가 역주행을 한 것으로 착각했다고 진술했다'고 설명했다."""

def change_inference_chat_format(input_text):
    return [
        # 입력문 요약을 위한 대화
        {"role": "user", "content": f"{input_text}"},
        {"role": "assistant", "content": """부산의 한 왕복 2차선 도로에서 역주행 사고로 배달
        오토바이 운전자인 고등학생이 숨지는 사고가 발생했다.
        유족은 '가해자가 사고 후 곧바로 신고하지 않고 늑장 대응해 피해를 키웠다'고 주장하고 있다.
        """},
        # 입력문 키워드 추출을 위한 대화
        {"role": "user", "content": "중요한 키워드 5개를 뽑아주세요."},
        {"role": "assistant", "content": ""}
    ]

prompt = change_inference_chat_format(input_text)

# prompt에 tokenizer 초기화 및 적용 
# 이로써 prompt의 text들은 토크나이저로 토큰단위로 분할되고 이는 vocab의 idx로 매핑
# 이는 모델이 텍스트를 해석할 수 있도록 vector로 바꿔주기 위함
inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
add_generation_prompt=True, return_tensors="pt").to(model.device)

# 토크나이저로 벡터화된 input을 모델이 해석하여 답변 (outputs도 역시 벡터)
outputs = model.generate(inputs_ids=inputs.to(model.device), max_new_tokens=256)
# 벡터였던 outputs를 text화
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# 요약 기능 확인

# input_text : 같은 기사 활용
def change_inference_chat_format(input_text):
    return [
        {"role": "user", "content": f"{input_text}"},
        {"role": "assistant", "content": "한국어 요약:\n"}
    ]

# chat_template 적용
prompt = change_inference_chat_format(input_text)

# 생성
inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
add_generation_prompt=True, return_tensors="pt").to(model.device)

outputs = model.generate(inputs, max_new_tokens=256, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# 키워드 추출 & 요약 기능 확인

# input_text : 위에 정의된 기사와 동일
def change_inference_chat_format(input_text):
    return [
        {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약하고,
            관련된 5개의 키워드를 추출해주세요:\n{input_text}"},
        {"role": "assistant", "content": ""},
    ]
prompt = change_inference_chat_format(input_text)

inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
add_generation_prompt=True, return_tensors="pt").to(model.device)

outputs = model.generate(inputs, max_new_tokens=256, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# 키워드 데이터 생성
# 원본 데이터는 original, summary만 제공하므로 키워드를 추출하도록 키워드 데이터 생성하기

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

# 키워드 추출하는 프롬프트
def key_word_prompt(input_text, summary_text):
    return [
        {"role": "user", "content": f"{input_text}"},
        {"role": "assistant", "content": f"{summary_text}"},
        {"role": "user", "content": "중요한 키워드 5개를 뽑아주세요."},
        {"role": "assistant", "content": ""}
    ]

# 각 batch의 데이터들의 keyword를 뽑아내는 프롬프트 작성
def extract_keyword_batch(batch):
    prompts = [key_word_prompt(original, summary) for original, summary in zip(batch["original"], batch["summary"])]

    generated_text = pipe(prompts, max_new_tokens=150, return_full_text=False)
    print(generated_text) # 얘의 형태를 봐야할 듯 key값들이 뭔지, attribute가 뭔지
    keywords = [gen_text[0]["generated_text"] for gen_text in generated_text]

    # 원래 데이터의 키값으로 키워드추가하고 저장
    batch["keyword"] = keywords
    return batch

# dataset에 keyword 열 추가
sample_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
sample_dataset = sample_dataset.map(extract_keyword_batch, batched=True, batch_size=20)

sample_dataset

In [ ]:
# 데이터 전처리

# 키워드와 요약 둘다 제공하는 프롬프트 생성
def chat_keyword_summary_format(example):
    return [
        {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약 및
            관련 키워드를 추출해주세요:\n{example["original"]}"},
        {"role": "assistant", "content": f"한국어 요약:{example["summary"]}\n
                키워드:{example["keywords"]}"}
    ]

formatted = tokenizer.apply_chat_template(
    chat_keyword_summary_format(sample_dataset[0]), tokenize=False
)

print(formatted)

In [ ]:
# apply_chat_template을 적용하고 토크나이저를 이용해 문자를 숫자로 변환하는 작업을 한번에 수행
EOS_TOKEN = tokenizer.eos_token # 그냥 tokenizer를 거치면 eos token이 없어서 직접 추가
def tokenize(element):
		# 토크나이저가 읽기 편하게 텍스트 -> 벡터화 (특수 토큰까지 적용해서 반환)
		formatted = tokenizer.apply_chat_template(
				chat_keyword_summary_format(element),
				tokenize=False) + EOS_TOKEN
		outputs = tokenizer(formatted) # output 벡터화된 토큰
		print(outputs) # input_ids와 attention_mask의 키값은 apply_chat_template에 의해 생기나?
		return {
				"input_ids": outputs["input_ids"],
				"attention_mask": outputs["attention_mask"],
		}

tokenized_sample_dataset = sample_dataset.map(tokenize)

In [ ]:
# 데이터셋 분리 및 콜레이터 설정

# 데이터셋 분리
tokenized_sample_dataset = tokenized_sample_dataset.train_test_split(
	test_size=0.1,
	seed=42
)
tokenized_sample_dataset

# 콜레이터 설정
response_template_ids = tokenizer.encode(
	"<start_of_turn>model\n",
	add_special_tokens=False
	)

collator = DataCollatorForCompletionOnlyLM(
	response_template_ids, tokenizer=tokenizer
)

In [ ]:
# 학습 파라미터 설정

wandb.init(project="gemma-2B-it-Full-Fine-Tuning", entity="Your_wandb_ID")

training_args = TrainingArguments(
    output_dir="./keywords_gemma_results",
    max_steps=800,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=0,
    weights_decay=0.01,
    learning_rate=2e-4,
    logging_dir="./logs",
    logging_steps=100,
    report_to="wandb",
)

In [ ]:
# 평가 메트릭 정의

bleu = evaluate.load("bleu")
acc = evaluate.load("accuracy")

def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        # 모델과 설정에 따라 logits에는 추가적인 텐서들이 포함될 수 있음
        # 예로 past_key_values 같은 것들이 있을 수 있지만, logits는 항상 첫 번째 요소임
        logits = logits[0]
    # 토큰 ID를 얻기 위해 argmax를 수행
    return logits.argmax(dim=-1)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds는 labels와 같은 형태를 가짐
    # preprocess_logits_for_metrics에서 argmax(-1)가 계산된 후의 결과
    # 하지만 우리는 labels를 한 칸 이동해야 함
    labels = labels[:, 1:] # 여긴 이해 필요
    preds = preds[:, :-1] 

    # -100은 DataCollatorForCompletionOnlyLM에서 사용되는 ignore_index의 기본값임
    mask = labels == -100
    # -100을 토크나이저가 디코드할 수 있는 값으로 대체합니다.
    labels[mask] = tokenizer.pad_token_id
    preds[mask] = tokenizer.pad_token_id

    # BLEU 점수는 텍스트를 입력으로 받기 때문에,
    # 토큰 ID에서 텍스트로 변환해야 됨 (디코딩)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # bleu score 계산 (생성 텍스트와 참조 텍스트 유사도 비교)
    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # accuracy는 정수 리스트를 입력으로 받음
    # 우리는 -100이 아닌 부분만 평가하고 싶으므로 마스크의 부정(~)을 사용
    accuracy = acc.compute(predictions=preds[~mask], references=labels[~mask])

    return {**bleu_score, **acuracy} # 둘 다 이중리스트로 쌓여있나봄 **로 값만 추출

In [ ]:
# Trainer 클래스
# 위에서 설정한 모델, 토크나이저, 데이터셋, 콜레이터, 평가지표를 하나로 묶어 학습을 지원 (pipeline 느낌)

trainer = Trainer(
    args=trainig_args,
    model=model,
    tokenizer=tokenizer,
    data_collator=collator,
    train_dataset=tokenized_sample_dataset["train"],
    eval_dataset=tokenized_sample_datsaet["test"],
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    callbacks=[WandbCallback()]
)


In [ ]:
# 파인튜닝한 모델 테스트
input_text = "부산의 한 왕복 2차선 도로에서 역주행 사고로 배달 오토바이 운전자인 고등학생이 숨지는 사고가 발생했다. 유족은 '가해자가 사고 후 곧바로 신고하지 않고 늑장 대응해 피해를 키웠다'고 주장하고 있다.\n11일 부산진경찰서는 교통사고처리특례법(교통사고처리법)상 업무상 과실치사 혐의로 지난 3일 A(59)씨를 검찰에 불구속 송치했다고 밝혔다. A씨는 교통사고처리법상 12대 중과실에 해당되는 '중앙선 침범'으로 역주행 교통사고를 일으킨 혐의를 받는다.\n경찰에 따르면 스포츠유틸리티차량(SUV) 운전자 A씨는 5월 19일 밤 11시 50분쯤 부산진구 가야고가교 밑 도로에서 중앙선을 넘어 역주행으로 140m를 달려 반대편 차선의 오토바이 운전자 조모(16)군을 들이받았다. 조군은 원동기장치자전거 면허를 취득한 상태였고 헬멧도 쓰고 있었지만 크게 다쳤다. 사고 당일 수술을 받았으나 얼마 후 2차 뇌출혈로 뇌사 판정이 내려졌고, 사고 발생 약 한 달 만인 지난달 16일 끝내 사망했다. 사고를 낸 A씨는 술을 마시거나 약물을 복용한 상태에서 운전하지는 않은 것으로 조사됐다. 경찰 관계자는 'A씨가 자신이 정주행을 하고 오토바이가 역주행을 한 것으로 착각했다고 진술했다'고 설명했다."

# 입력된 기사를 요약하고, 키워드를 추출하는 역할할
def get_chat_format(input_text):
    return [
        {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약 및 관련 키워드를 추출해주세요:\n{input_text}"},
        {"role": "assistant", "content": "한국어 요약:\n키워드:"}
    ]

# 입력 기사를 대화형 prompt로 변환하는 역할
def change_inference_chat_format(input_text):
    return [
        {"role": "user", "content": f"{input_text}"},
        {"role", "assistant", "content": ""}
    ]

prompt = change_inference_chat_format(input_text) # input -> prompt로 변환

# tokenizer 초기화 및 적용
inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
    add_generation_prompt=True, return_tensor="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=512, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))